# NETWORK PART WITH MOTION DETECTION

This program further processes the output from the `ms_input` file, converts them to spikes and contains the Reichardt detector system. 
It only processes 1D left-rightward motion.

It includes midget cell setups, in order to investigate their potential behaviour. However it could be omitted from the code easily.

In [ ]:
#THIS IS THE PART OF THE NEURAL NETWORK. IT TAKES THE VALUES TO CALCULATE THE MEMBRANE POTENTIALS OF THE INPUT NEURONS AND THEN CALCULATES THE NETWORK OUTPUT 
#the unit length in space is 1arcmin!
import pylab as pyl
import numpy as np
import cv2
import sys
import os
import matplotlib.pyplot as plt
import nest
import datetime
import nest.raster_plot
import nest.topology as tp
from microsaccades_functions import *

pgf_with_rc_fonts = {"font.family": "serif","font.serif": [],"font.sans-serif": []}
plt.rcParams.update(pgf_with_rc_fonts)

Settings for `NEST`, especially to enable randomization. This is used then for the random input current $I_E$.

In [ ]:
nest.ResetKernel()   # since we run the script multiple times
#to get different poisson outputs

msd = int(np.random.normal(5000,1000,1)[0])  #master seed
nest.SetKernelStatus({'local_num_threads' : 4})
n_vp = nest.GetKernelStatus('total_num_virtual_procs')
msdrange1 = range(msd, msd+n_vp)
pyrngs = [np.random.RandomState(s) for s in msdrange1]
msdrange2 = range(msd+n_vp+1, msd+1+2*n_vp)
nest.SetKernelStatus({'grng_seed': msd+n_vp,
                      'rng_seeds': msdrange2})

I_E=355.

def set_I_e_random(layer):
    r = nest.GetNodes(layer)[0]
    node_info=nest.GetStatus(r)
    localnodes=[(ni['global_id'],ni['vp']) for ni in node_info if ni['local']]
    for gid, vp in localnodes:
        nest.SetStatus([gid], {'I_e' : I_E+pyrngs[vp].uniform(-2.,2.)})

Setting the main parameters / reading them from the input script.

In [ ]:
#necessary paramater definitions
#frames = 100 #replaced by motion detectors only

#----------------------------------------------------------------------------INPUT-RATES-FROM-MS_INPUT

extent = 121.
delay = 30. #15. #30. # speed of point in poletti 2010 -> maybe increase a bit for more reacton later on

t_start = 0
t_end = 630 #630 #1000

weight = 40.
weight_std = 0.5
#I_E = 410.

mdw=4.0

if len(sys.argv)==11:
    sim_title = sys.argv[1]
    sim_title_2 = sys.argv[2]
    sim_nr = sys.argv[3]
    handle_name = sys.argv[4]
    extent_x = float(sys.argv[5])
    extent_y = float(sys.argv[6])
    exp_nr = sys.argv[7]
    cond_nr = sys.argv[8]
    sim_length = int(float(sys.argv[9]))
else:
    sim_title = sys.argv[1]
    sim_nr = sys.argv[2]
    handle_name = sys.argv[3]
    extent_x = float(sys.argv[4])
    extent_y = float(sys.argv[5])
    exp_nr = sys.argv[6]
    cond_nr = sys.argv[7]
    sim_length = int(float(sys.argv[8]))
    sim_title_2 = ''
	
t_end=sim_length

vel = 0.
center_x = extent_x/2.
center_y = extent_y/2.

Loading the data from `ms_input` and converting them to rates. The scaling factors `mmr` and `pmr` are chosen such that spikes are produced with maximal rates between 60-100 spikes/s.

In [ ]:
#transfering potentials into poisson rates
m_file = open('data/'+str(sim_title)+'/'+str(sim_nr)+'/midget_rates_'+str(handle_name)+'_on.data','r+')
m_data = np.load(m_file)   
m_file.close()

p_file = open('data/'+str(sim_title)+'/'+str(sim_nr)+'/parasolic_rates_'+str(handle_name)+'_on.data','r+') 
p_data = np.load(p_file)  
p_file.close()

midget_rates=poissonRateMidgets(m_data)
parasolic_rates=poissonRateParasols(p_data)

#setting the midget / parasol multipication constant converting the output to external current
#mmr might need adaptiation for different conditions, but the currently chosen is optimized for the current output.
mmr = 500000. #10000000. #500000. #330000. 
pmr = 480000.
     
#for current 
mrs = []
prs=[]

Now, the neuron models needed for the simulation are created with `NEST`.

In [ ]:
#-----------------------------------------------------------------------------------------NETWORK-PART
#---------------------------------------------------------------------------INITIALIZE-POISSON-NEURONS

#create all newly needed models
nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_mp") #,{"I_e" : I_E})
nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_i",{"I_e" : I_E})
#nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_r",{"I_e" : 374.0})
nest.CopyModel("spike_detector", "my_spike_detector",{"withgid": True,
                                                      "withtime": True,
                                                      "to_memory": True,})

nest.CopyModel('static_synapse','ex',{'weight': 1.0})

In [ ]:
#----------------------------------------------------------------------------------------CREATE-LAYERS

#get grid data form previous simulation
gm_file = open('data/'+str(sim_title)+str(sim_title_2)+'/'+str(sim_nr)+'/m_pos_'+handle_name+'.data','r+')
gm_data = np.load(gm_file)  
gm_file.close()
gm_data = gm_data.tolist()

gp_file = open('data/'+str(sim_title)+str(sim_title_2)+'/'+str(sim_nr)+'/p_pos_'+handle_name+'.data','r+')
gp_data = np.load(gp_file)  
gp_file.close()
gp_data = gp_data.tolist()

gm_pos=[]
#positions for the reichardt detectors, which have to be centered
gm_r_0_pos=[]

gp_pos=[]
#positions for the reichardt detectors, which have to be centered
gp_r_0_pos=[]

For the midget as well as the parasol cell layers, noise is added (optional).

In [ ]:
m_noise=[[np.random.normal(0,10,t_end) for j in range(len(gm_data[0]))] for i in range(len(gm_data))]
midget_rates=mmr*midget_rates+m_noise

p_noise=[[np.random.normal(0,10,t_end) for j in range(len(gp_data[0]))] for i in range(len(gp_data))]
parasolic_rates=pmr*parasolic_rates+p_noise

The positions of the parasol and midget cells as well as the motion detectors are determined

In [ ]:
for i in range(len(gp_data)):
    for j in range(len(gp_data[0])):
        prs+=[parasolic_rates[i][j]]
        gp_pos+=[[0.5*gp_data[i][j][0]+0.25,0.5*gp_data[i][j][1]+0.25]]
        gp_r_0_pos+=[[0.5*gp_data[i][j][0]+2.25,0.5*gp_data[i][j][1]+0.25]]
        #gp_r_60_pos+=[[0.5*gp_data[i][j][0]+1.25,0.5*gp_data[i][j][1]+0.25+1.732]]

The layers are created in `NEST`. `vel2` indicates velocity tuning two the double velocity.

In [ ]:
midgets = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'iaf_psc_alpha_mp', 'edge_wrap': True})
parasolic = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'iaf_psc_alpha_mp', 'edge_wrap': True})

#motion detectors - these are direction dependent
m_reichardt_0_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})
m_reichardt_0_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})

m_reichardt_0_vel2_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})
m_reichardt_0_vel2_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})

p_reichardt_0_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})
p_reichardt_0_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})
p_reichardt_0_vel2_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})
p_reichardt_0_vel2_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'iaf_psc_alpha_i', 'edge_wrap': True})

#layers for output -> spike detectors
out_m = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'my_spike_detector'})
out_p = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'my_spike_detector'})

out_m_r_0_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'my_spike_detector'})
out_m_r_0_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'my_spike_detector'})

out_m_r_0_vel2_left = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'my_spike_detector'})
out_m_r_0_vel2_right = tp.CreateLayer({'extent' : [extent_x,extent_y], 'center' : [center_x,center_y], 'positions' : gm_pos, 'elements': 'my_spike_detector'})

out_p_r_0_left = tp.CreateLayer({'extent' : [extent_x+4.,extent_y+4.], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'my_spike_detector'})
out_p_r_0_right = tp.CreateLayer({'extent' : [extent_x+4.,extent_y+4.], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'my_spike_detector'})

out_p_r_0_vel2_left = tp.CreateLayer({'extent' : [extent_x+4.,extent_y+4.], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'my_spike_detector'})
out_p_r_0_vel2_right = tp.CreateLayer({'extent' : [extent_x+4.,extent_y+4.], 'center' : [center_x,center_y], 'positions' : gp_pos, 'elements': 'my_spike_detector'})


Connecting the layers.

In [ ]:
#-------------------------------------------------------------------------------------------------CREATE-CONNECTIONS
#connections to left/right half of Reichardt detector
V_input_conndict = {'connection_type' : 'convergent', 'synapse_model': 'stdp_synapse', 'mask' : {'rectangular' : {'lower_left' : [-0.1,-0.1], 'upper_right' : [0.1,0.1]}}}

m_r_0_left_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': {'uniform': {'min': weight-weight_std, 'max': weight+weight_std}}, 'mask' : {'rectangular' : {'lower_left' : [-(0.25*mdw+0.1),-0.1], 'upper_right' : [0.1,0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}
m_r_0_right_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': {'uniform': {'min': weight-weight_std, 'max': weight+weight_std}}, 'mask' : {'rectangular' : {'lower_left' : [-0.1,-0.1], 'upper_right' : [(0.25*mdw+0.1),0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}
m_r_0_vel2_left_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': {'uniform': {'min': weight-weight_std, 'max': weight+weight_std}}, 'mask' : {'rectangular' : {'lower_left' : [-(0.25*mdw+0.1),-0.1], 'upper_right' : [0.1,0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : 0.5*delay}}}
m_r_0_vel2_right_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': {'uniform': {'min': weight-weight_std, 'max': weight+weight_std}}, 'mask' : {'rectangular' : {'lower_left' : [-0.1,-0.1], 'upper_right' : [(0.25*mdw+0.1),0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : 0.5*delay}}}

p_r_0_left_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': weight, 'mask' : {'rectangular' : {'lower_left' : [-(mdw+0.1),-0.1], 'upper_right' : [0.1,0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}
p_r_0_right_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': weight, 'mask' : {'rectangular' : {'lower_left' : [-0.1,-0.1], 'upper_right' : [(mdw+0.1),0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}
p_r_0_vel2_left_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': weight, 'mask' : {'rectangular' : {'lower_left' : [-(mdw+0.1),-0.1], 'upper_right' : [0.1,0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}
p_r_0_vel2_right_conndict = {'connection_type' : 'convergent', 'synapse_model': 'ex', 'weights': weight, 'mask' : {'rectangular' : {'lower_left' : [-0.1,-0.1], 'upper_right' : [(mdw+0.1),0.1]}}, 'delays' : {'linear' : {'c' : .1, 'a' : delay}}}


out_conndict = {'connection_type' : 'convergent', 'mask' : {'rectangular' : {'lower_left' : [-0.2,-0.2], 'upper_right' : [0.2,0.2]}}}

#connect them

tp.ConnectLayers(midgets,m_reichardt_0_right,m_r_0_right_conndict)
tp.ConnectLayers(midgets,m_reichardt_0_left,m_r_0_left_conndict)
tp.ConnectLayers(midgets,m_reichardt_0_vel2_right,m_r_0_vel2_right_conndict)
tp.ConnectLayers(midgets,m_reichardt_0_vel2_left,m_r_0_vel2_left_conndict)

tp.ConnectLayers(parasolic,p_reichardt_0_right,p_r_0_right_conndict)
tp.ConnectLayers(parasolic,p_reichardt_0_left,p_r_0_left_conndict)

tp.ConnectLayers(parasolic,p_reichardt_0_vel2_right,p_r_0_vel2_right_conndict)
tp.ConnectLayers(parasolic,p_reichardt_0_vel2_left,p_r_0_vel2_left_conndict)

tp.ConnectLayers(midgets,out_m,out_conndict)
tp.ConnectLayers(parasolic,out_p,out_conndict)

tp.ConnectLayers(m_reichardt_0_left,out_m_r_0_left,out_conndict)
tp.ConnectLayers(m_reichardt_0_right,out_m_r_0_right,out_conndict)
tp.ConnectLayers(p_reichardt_0_left,out_p_r_0_left,out_conndict)
tp.ConnectLayers(p_reichardt_0_right,out_p_r_0_right,out_conndict)

tp.ConnectLayers(m_reichardt_0_vel2_left,out_m_r_0_vel2_left,out_conndict)
tp.ConnectLayers(m_reichardt_0_vel2_right,out_m_r_0_vel2_right,out_conndict)
tp.ConnectLayers(p_reichardt_0_vel2_left,out_p_r_0_vel2_left,out_conndict)
tp.ConnectLayers(p_reichardt_0_vel2_right,out_p_r_0_vel2_right,out_conndict)


The actual simulation. As updates of the external current need to be applied every step, we iterate through the steps, simulation always just for the duration of one step.

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------SIMULATION

#for updates
MIDs = nest.GetNodes(midgets)
PIDs = nest.GetNodes(parasolic)
SIE = 374.7
STDI = 0.45

#set the end time depending on the current conditions...
for f in range(t_start,t_start+420): #t_end):#frames):
    print f
    #reset rates
    for n in range(len(MIDs[0])):
        qr = mrs[n][f]
        #spontaneous firing rate, only if wished to be included
        '''
        if qr < 375.:
            qr = SIE+np.random.normal(0,STDI,1)[0]
        if f<5:
            qr = 374.+float(np.random.normal(0,1.7,1)[0])
        '''
        nest.SetStatus([MIDs[0][n]], {'I_e': qr})
    
    #run simulation
    nest.Simulate(1)

Save all the values needed for evaluation and further processing in order to suppress global motion signals.

`save_spikes` - saves spikes with positions and times for each layer.

`save_spikes_200` - does the same, but starts only after 200ms of simulated time, if one wishes to leave out initiation effects.

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------SAVING-AND-PRINTING-THE-OUTPUT

max_evs_list = []
max_evs_list_200 = []
all_vel2_spikes = 0
all_vel2_spikes_200 = 0
m_all_vel2_spikes = 0
p_all_vel2_spikes = 0
m_all_vel2_spikes_200 = 0
p_all_vel2_spikes_200 = 0
all_spikes = 0
all_spikes_200 = 0
m_all_spikes = 0
p_all_spikes = 0
m_all_spikes_200 = 0
p_all_spikes_200 = 0

m_all_vel2_times=[]
p_all_vel2_times=[]
m_left_vel2_times=[]
m_right_vel2_times=[]
p_left_vel2_times=[]
p_right_vel2_times=[]
m_all_times=[]
p_all_times=[]
m_left_times=[]
m_right_times=[]
p_left_times=[]
p_right_times=[]
m_ud_times=[]
p_ud_times=[]

m_all_vel2_times_200=[]
p_all_vel2_times_200=[]
m_left_vel2_times_200=[]
m_right_vel2_times_200=[]
p_left_vel2_times_200=[]
p_right_vel2_times_200=[]
m_all_times_200=[]
p_all_times_200=[]
m_left_times_200=[]
m_right_times_200=[]
p_left_times_200=[]
p_right_times_200=[]
m_ud_times_200=[]
p_ud_times_200=[]

def save_spikes(layer_name,layer,mel,asl,tl):
    directory = '/data/'+str(sim_title)+'/network/'+str(sim_nr)
    sp_file = open(directory+'/'+layer_name+'_'+handle_name+'.data','w+')
    n_evs_l=[0]
    times = []
    for n in range(len(layer[0])):
        n_evs = nest.GetStatus([layer[0][n]],"n_events")[0]
        if n_evs>0:
            #print layer_name+' '+str(layer[0][n])+' '+str(n_evs)+' '+str(nest.GetStatus([layer[0][n]],"events")[0]["times"])
            t_evs = nest.GetStatus([layer[0][n]],"events")[0]["times"]
            sp_file.write(str(layer[0][n])+'\t'+str(nest.GetStatus([layer[0][n]],"events")[0]["times"])+'\n')
            for t in t_evs:
                times+=[t]
            n_evs_l+=[n_evs]
    mel+=[max(n_evs_l)]
    #if layer_name != ('spikes_midgets' or 'spikes_parasols'):
    asl+=sum(n_evs_l) 
    tl=set(times)
    sp_file.close()
    print layer_name
    return 0

def save_spikes_200(layer_name,layer,mel200,asl200,tl200):
    directory = '/data/'+str(sim_title)+'/network/'+str(sim_nr)
    sp_file = open(directory+'/'+layer_name+'_200_'+handle_name+'.data','w+')
    n_evs_200_l=[0]
    times=[]
    for n in range(len(layer[0])):
        n_evs = nest.GetStatus([layer[0][n]],"n_events")[0]
        if n_evs>0:
            t_evs = nest.GetStatus([layer[0][n]],"events")[0]["times"]
            t_evs_200 = [t_ev for t_ev in t_evs if t_ev>200.]
            if len(t_evs_200)>0:
                #print layer[0][n],t_evs_200
                sp_file.write(str(layer[0][n])+'\t'+str(t_evs_200)+'\n')
                n_evs_200_l+=[len(t_evs_200)]
                for t in t_evs:
                    times+=[t]
    mel200+=[max(n_evs_200_l)]
    #if layer_name != ('spikes_midgets' or 'spikes_parasols'):
    asl200+=sum(n_evs_200_l) 
    tl200=set(times)
    sp_file.close()
    print layer_name + '200'
    
    return 0

GID_file = open('/data/'+str(sim_title)+'/network/'+str(sim_nr)+'/GID_info.txt','w+')

#midget
layerIDs = nest.GetNodes(out_m)
GID_file.write('midgets \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_midgets',layerIDs,max_evs_list,all_spikes,m_all_times)
save_spikes_200('spikes_midgets',layerIDs,max_evs_list_200,all_spikes_200,m_all_times_200)
#parasolic
layerIDs = nest.GetNodes(out_p)
GID_file.write('parasols \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_parasols',layerIDs,max_evs_list,all_spikes,p_all_times)
save_spikes_200('spikes_parasols',layerIDs,max_evs_list_200,all_spikes_200,p_all_times_200)

#midget rightward motion detectors
layerIDs = nest.GetNodes(out_m_r_0_left)
GID_file.write('m_0_left \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_m_0_left',layerIDs,max_evs_list,m_all_spikes,m_left_times)
save_spikes_200('spikes_m_0_left',layerIDs,max_evs_list_200,m_all_spikes_200,m_left_times_200)
#midget leftward motion detectors
layerIDs = nest.GetNodes(out_m_r_0_right)
GID_file.write('m_0_right \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_m_0_right',layerIDs,max_evs_list,m_all_spikes,m_right_times)
save_spikes_200('spikes_m_0_right',layerIDs,max_evs_list_200,m_all_spikes_200,m_right_times_200)
#midget rightward motion detectors
layerIDs = nest.GetNodes(out_m_r_0_vel2_left)
GID_file.write('m_0_vel2_left \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_m_0_vel2_left',layerIDs,max_evs_list,m_all_vel2_spikes,m_left_vel2_times)
save_spikes_200('spikes_m_0_vel2_left',layerIDs,max_evs_list_200,m_all_vel2_spikes_200,m_left_vel2_times_200)
#midget leftward motion detectors
layerIDs = nest.GetNodes(out_m_r_0_vel2_right)
GID_file.write('m_0_vel2_right \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_m_0_vel2_right',layerIDs,max_evs_list,m_all_vel2_spikes,m_right_vel2_times)
save_spikes_200('spikes_m_0_vel2_right',layerIDs,max_evs_list_200,m_all_vel2_spikes_200,m_right_vel2_times_200)

#parasolic rightward motion detectors
layerIDs = nest.GetNodes(out_p_r_0_left)
GID_file.write('p_0_left \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_p_0_left',layerIDs,max_evs_list,p_all_spikes,p_left_times)
save_spikes_200('spikes_p_0_left',layerIDs,max_evs_list_200,p_all_spikes_200,p_left_times_200)
#parasolic leftward motion detectors
layerIDs = nest.GetNodes(out_p_r_0_right)
GID_file.write('p_0_right \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_p_0_right',layerIDs,max_evs_list,p_all_spikes,p_right_times)
save_spikes_200('spikes_p_0_right',layerIDs,max_evs_list_200,p_all_spikes_200,p_right_times_200)

#parasolic rightward motion detectors
layerIDs = nest.GetNodes(out_p_r_0_vel2_left)
GID_file.write('p_0_vel2_left \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_p_0_vel2_left',layerIDs,max_evs_list,p_all_vel2_spikes,p_left_vel2_times)
save_spikes_200('spikes_p_0_vel2_left',layerIDs,max_evs_list_200,p_all_vel2_spikes_200,p_left_vel2_times_200)
#parasolic leftward motion detectors
layerIDs = nest.GetNodes(out_p_r_0_vel2_right)
GID_file.write('p_0_vel2_right \t'+str(layerIDs[0][0])+'\t'+str(layerIDs[0][len(layerIDs[0])-1])+'\n')
save_spikes('spikes_p_0_vel2_right',layerIDs,max_evs_list,p_all_vel2_spikes,p_right_vel2_times)
save_spikes_200('spikes_p_0_vel2_right',layerIDs,max_evs_list_200,p_all_vel2_spikes_200,p_right_vel2_times_200)

GID_file.close()

print 'finished'

#reset values just to make sure

midgets = 0
parasols = 0
m_reichardt_0_left = 0
m_reichardt_0_right = 0
m_reichardt_60_up = 0
m_reichardt_60_down = 0
m_reichardt_120_up = 0
m_reichardt_120_down = 0
p_reichardt_0_left = 0
p_reichardt_0_right = 0
p_reichardt_60_up = 0
p_reichardt_60_down = 0
p_reichardt_120_up = 0
p_reichardt_120_down = 0

out_m = 0
out_p = 0
out_m_r_0_left = 0
out_m_r_0_right = 0
out_m_r_60_down = 0
out_m_r_60_up = 0
out_m_r_120_down = 0
out_m_r_120_up = 0
out_p_r_0_left = 0
out_p_r_0_right = 0
out_p_r_60_up = 0
out_p_r_60_down = 0
out_p_r_120_up = 0
out_p_r_120_down = 0

sys.exit()